In [1]:
import pandas as pd
train_df = pd.read_json('../input/train.json')
test_df = pd.read_json('../input/test.json')
print(train_df.head())

                                              band_1  \
0  [-27.878360999999998, -27.15416, -28.668615, -...   
1  [-12.242375, -14.920304999999999, -14.920363, ...   
2  [-24.603676, -24.603714, -24.871029, -23.15277...   
3  [-22.454607, -23.082819, -23.998013, -23.99805...   
4  [-26.006956, -23.164886, -23.164886, -26.89116...   

                                              band_2        id inc_angle  \
0  [-27.154118, -29.537888, -31.0306, -32.190483,...  dfd5f913   43.9239   
1  [-31.506321, -27.984554, -26.645678, -23.76760...  e25388fd   38.1562   
2  [-24.870956, -24.092632, -20.653963, -19.41104...  58b2aaa0   45.2859   
3  [-27.889421, -27.519794, -27.165262, -29.10350...  4cfc3a18   43.8306   
4  [-27.206915, -30.259186, -30.259186, -23.16495...  271f93f4   35.6256   

   is_iceberg  
0           0  
1           0  
2           1  
3           0  
4           0  


In [3]:
import numpy as np

from scipy.ndimage import laplace, sobel
from scipy.stats import kurtosis, skew

def get_img_feat(img):
    feats = [np.mean(img),np.std(img),np.median(img),np.max(img),np.min(img)]
    
    return feats
    
def get_other_feat(df):
    feats = []
    for i, row in df.iterrows():
        tmp_feat = []
        img1 = np.array(row['band_1']).reshape(75, 75).astype('float32')
        img2 = np.array(row['band_2']).reshape(75, 75).astype('float32')
        if row['inc_angle'] == 'na':
            ang = 0
        else:
            ang = float(row['inc_angle'])
        tmp_feat = get_img_feat(img1) + get_img_feat(img2) + [ang]
        
        
        feats.append(tmp_feat)
        
    return np.array(feats).astype('float32')

train_feat = get_other_feat(train_df)
test_feat = get_other_feat(test_df)

In [4]:
print(train_feat[:2])  

[[-27.91104507   2.76453733 -27.87992096  -0.213149   -38.21137619
  -29.9101162    2.381284   -30.00784683 -11.2521534  -41.13591766
   43.9239006 ]
 [-13.56655312   3.14253163 -13.65419865  12.57040882 -23.12530899
  -25.35910606   2.93409801 -25.48612976   0.044052   -34.76583099
   38.15620041]]


In [5]:
train_feat.shape

(1604, 11)

In [6]:
train_feat.dtype

dtype('float32')

In [7]:
import pickle
with open('../features/other_feat.pkl','wb') as fout:
    pickle.dump([train_feat,test_feat],fout)